<a href="https://colab.research.google.com/github/barkhaneum/Transformer_study/blob/main/chapter_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -U datasets # load_dataset 대해 오류가 나면 업데이트
!pip install evaluate # load_metrics대신 사용
!pip install sacrebleu
!pip install rouge_score

from datasets import load_dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [11]:
sample = dataset['train'][1] # highlights, article, id
print(f"""기사 (500개 문자 발췌, 총 길이 : {len(sample['article'])} ) :""")
print(sample['article'][:500]) # 기사 내용 발췌
print(f'\n요약 ( 길이 : {len(sample["highlights"])}) :')
print(sample['highlights'])


기사 (500개 문자 발췌, 총 길이 : 4051 ) :
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s

요약 ( 길이 : 281) :
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


In [12]:
sample_text = dataset['train'][1]['article'][:2000] # 기사 내용 발췌
summaries = {}

In [13]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt_tab")
nltk.download("punkt")

string = "The U.S. are a country. The U.N. is an organization."
sent_tokenize(string)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['The U.S. are a country.', 'The U.N. is an organization.']

In [14]:
def three_sentence_summary(text):
  return "\n".join(sent_tokenize(text)[:3]) # 텍스트중 맨처음 문장 세개를 선택

summaries["baseline"] = three_sentence_summary(sample_text)

In [15]:
import torch
from transformers import pipeline, set_seed

device = 0 if torch.cuda.is_available() else -1
set_seed(42)

pipe = pipeline("text-generation", model = "gpt2-xl", device=device)
gpt2_query = sample_text + "\nTl;DR:\n"

pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces = True)
summaries["gpt2"] = "\n".join(sent_tokenize(pipe_out[0]['generated_text'][len(gpt2_query):]))

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [16]:
pipe = pipeline("summarization", model = "t5-large")
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cuda:0


In [17]:
pipe = pipeline("summarization", model = "facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [18]:
pipe = pipeline("summarization", model = "google/pegasus-cnn_dailymail")
pipe_out = pipe(sample_text)
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [19]:
print("GROUND TRUTH")
print(dataset["train"][1]["highlights"])
print("")

for model_name in summaries:
  print(model_name.upper())
  print(summaries[model_name])
  print("")

GROUND TRUTH
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .

BASELINE
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."

GPT2
The mentally ill aren't being properly treated and are being housed in an environment that is not conducive to their care.
The story of the mentally ill in the United States is a story of how we treat them.
We see them

# 텍스트 생서 모델 평가

## 1. BLEU 평가

In [20]:

import evaluate
bleu_metric = evaluate.load("sacrebleu")
rouge_metric = evaluate.load("rouge")

In [21]:
import pandas as pd
import numpy as np

bleu_metric.add(
    prediction = "the the the the the the", reference = ["the cat is on the mat"])
results = bleu_metric.compute(smooth_method = "floor", smooth_value = 0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient = "index", columns = ["Value"])

,Value
score,0.0
counts,"[2, 0, 0, 0]"
totals,"[6, 5, 4, 3]"
precisions,"[33.33, 0.0, 0.0, 0.0]"
bp,1.0
sys_len,6
ref_len,6


In [22]:
bleu_metric.add(
    prediction = "the cat is on mat", reference = ["the cat is on the mat"])
results = bleu_metric.compute(smooth_method = "floor", smooth_value = 0)
results["precisions"] = [ np.round(p, 2) for p in results["precisions"] ]
pd.DataFrame.from_dict(results, orient = "index", columns = ["Value"])

,Value
score,57.893007
counts,"[5, 3, 2, 1]"
totals,"[5, 4, 3, 2]"
precisions,"[100.0, 75.0, 66.67, 50.0]"
bp,0.818731
sys_len,5
ref_len,6


## 2. ROUGE 평가

In [23]:

reference = dataset["train"][1]["highlights"]
records = []
rouge_names = ["rouge1", "rouge2","rougeL", "rougeLsum"]

for model_name in summaries:
  rouge_metric.add(prediction = summaries[model_name], reference  = reference )
  score = rouge_metric.compute()
  rouge_dict = {rn: score[rn] for rn in rouge_names}
  records.append(rouge_dict)
pd.DataFrame.from_records(records, index = summaries.keys())

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.365079,0.145161,0.206349,0.285714
gpt2,0.187050,0.050725,0.115108,0.187050
t5,0.382979,0.130435,0.255319,0.382979
bart,0.475248,0.222222,0.316832,0.415842
pegasus,0.500000,0.325581,0.431818,0.477273


## CNN/DailyMail 데이터셋에서 PEGASUS 평가하기

In [24]:
def evaluate_summaries_baseline(dataset, metric,
                                column_text = "article",
                                column_summary = "highlights"):
  summaries = [three_sentence_summary(text) for text in dataset[column_text]]
  metric.add_batch(predictions = summaries, references = dataset[column_summary])
  score = metric.compute()
  return score

In [25]:
test_sampled = dataset["test"].shuffle(seed = 42).select(range(1000))

score = evaluate_summaries_baseline(test_sampled, rouge_metric)
rouge_dict = {rn: score[rn] for rn in rouge_names}
pd.DataFrame.from_dict(rouge_dict, orient = "index", columns=["baseline"]).T

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.389276,0.171296,0.245061,0.354239


In [26]:
from tqdm import tqdm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

def chunk(list_of_elements, batch_size):
  """list_of_elements로부터 batch_size 크기의 청크를 연속적으로 생성함"""
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i : i + batch_size]

def evaluate_summaries_pegasus(dataset, metric, model, tokenizer,
                               batch_size = 16, device = device,
                               column_text = "article",
                               column_summary = "highlights"):
  article_batches = list(chunk(dataset[column_text], batch_size))
  target_batches = list(chunk(dataset[column_summary], batch_size))

  for article_batch, target_batch in tqdm(
      zip(article_batches, target_batches), total = len(article_batches)):

      inputs = tokenizer(article_batch, max_length = 1024, truncation = True,
                         padding = "max_length", return_tensors = "pt")

      summaries = model.generate(input_ids = inputs["input_ids"].to(device),
                                 attention_mask = inputs["attention_mask"].to(device),
                                 length_penalty = 0.8, num_beams = 8, max_length = 128)

      decoded_summaries = [tokenizer.decode(s, skip_special_tokens = True,
                                            clean_up_tokenization_spaces = True)
                           for s in summaries]

      decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
      metric.add_batch(predictions = decoded_summaries, references = target_batch)

  score = metric.compute()
  return score



In [27]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
score = evaluate_summaries_pegasus(test_sampled, rouge_metric, model, tokenizer, batch_size = 8)
rouge_dict = {rn: score[rn] for rn in rouge_names}
pd.DataFrame(rouge_dict, index = ["pegasus"])

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 125/125 [1:08:50<00:00, 33.05s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.434236,0.216516,0.311725,0.37404


In [4]:
# samsum 데이터셋
dataset_samsum = load_dataset("knkarthick/samsum")
split_length = [len(dataset_samsum[split]) for split in dataset_samsum]

print(f"분할 크기: {split_length}")
print(f"특성: {dataset_samsum['train'].column_names}")
print("\n대화 : ")
print(dataset_samsum["test"][0]["dialogue"])
print("\n요약 : ")
print(dataset_samsum["test"][0]["summary"])

README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

분할 크기: [14732, 818, 819]
특성: ['id', 'dialogue', 'summary']

대화 : 
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

요약 : 
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


In [5]:
pipe_out = pipe(dataset_samsum["test"][0]["dialogue"])
print("요약 : ")
print(pipe_out[0]["summary_text"].replace(" .<n>", ".\n"))

NameError: name 'pipe' is not defined

In [41]:
score = evaluate_summaries_pegasus(dataset_samsum["test"], rouge_metric, model,
                                   tokenizer, column_text = "dialogue",
                                   column_summary = "summary",  batch_size = 8)
rouge_dict = {rn: score[rn] for rn in rouge_names}
pd.DataFrame(rouge_dict, index = ["pegasus"])

#  CNN/DailyMail데이터셋이 SAMSum과 다르기떄문에 좋은 결과가 나오진 않음
# 데이터셋을 훈련전 평가 파이프라인을 통해 훈련이 성공적인지 평가가 가능함
# 이 데이터셋에서 모델을 미세튜닝하면 ROUGE 점수가 향상되어야함

100%|██████████| 103/103 [52:11<00:00, 30.40s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.293241,0.086936,0.227665,0.227913


In [6]:
# samSUM데이터를 fine-tuning으로 훈련하기위한 데이터셋 분포 시각화
d_len = [len(tokenizer(x)["input_ids"]) for x in dataset_samsum["train"]["dialogue"]]
s_len = [len(tokenizer(x)["input_ids"]) for x in dataset_samsum["train"]["summary"]]

fig, axes = plt.subplots(1, 2, figsize = (10, 3.5), sharex = True)
axes[0].hist(d_len, bins = 20, color = "C0", edgecolor = "C0")
axes[0].set_title("Dialogue Token Length")
axes[0].set_xlabel("Length")
axes[0].set_ylabel("Count")
axes[1].hist(s_len, bins = 20, color = "C0", edgecolor = "C0")
axes[1].set_title("Summary Token Length")
axes[1].set_xlabel("Length")
plt.tight_layout()
plt.show()

NameError: name 'tokenizer' is not defined

In [ ]:
def convert_examples_to_features(example_batch):
  input_encodings = tokenizer(example_batch["dialogue"], max_length = 1024, truncation = True)

  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch["summary"], max_length = 128, truncation = True)

  return {"input_ids": input_encodings["input_ids"],
          "attention_mask" : input_encodings["attention_mask"],
          "labels":target_encodings["input_ids"]}

dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)
columns = ["input_ids", "labels", "attention_mask"]
dataset_samsum_pt.set_format(type = "torch", columns = columns)

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = "pegasus-samsum", num_train_epochs = 1, warmup_steps = 500,
    per_device_train_batch_size = 1, per_device_train_batch_size = 1,
    weight_decay = 0.01, logging_steps = 10, push_to_hub = True,
    evaluation_strategy = "steps", eval_steps = 500, save_steps = 1e6,
    gradient_accumulation_steps = 16)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
trainer = Trainer(model = model, args = training_args,
                  tokenizer = tokenizer, data_collator = seq2seq_data_collator,
                  train_dataset = dataset_samsum_pt["train"],
                  eval_dataset = dataset_samsum_pt["validation"])

In [ ]:
trainer.train()
score = evaluate_summaries_pegasus(dataset_samsum["test"], rouge_metric, trainer.model, tokenizer, batch_size = 2, device = device,
                                   column_text = "dialogue", column_summary = "summary")

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index = [f"pegasus"])

In [ ]:
trainer.push_to_hub("training complete!")

# 훈련된 모델을 바탕으로 대화 요약 생성하기

In [ ]:
gen_kwargs = {"length_penalty": 0.8, "num_beams": 8, "max_length": 128}
sample_text = dataset_samsum["test"][0]["dialogue"]
reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model = "[사용자이름]/pegasus-samsum")

print("대화:")
print("sample_text")
print("\n참조 요약 : ")
print(reference)
print("\n모델 요약 : ")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

In [ ]:
custom_dialogue = """\
Thom : Hi guys, have you heard of transformers?
Lewis : Yes, I used them recently!
leandro : Indeed, there is a great libary by Hugging Face.
Thom : I know, I helped build it ;)
Lewis : Cool, meybe we should write a book about it. What do you think?
Leandro : Great idea, how hard can it be?!
Thom :  I am in!
Lewis : Awesome, let's do it together!
"""

print(pipe(custom_dialouge, **gen_kwargs)[0]["summary_text"])